# Prerrequisites

Installing Spark and Apache Kafka Library in VM


---



In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7.tgz
!tar xf spark-3.0.1-bin-hadoop2.7.tgz
!pip install -q findspark==1.3.0
!pip install py4j==0.10.9

# For plotting
!pip install folium
!pip install plotly

     |████████████████████████████████| 204kB 7.1MB/s 


In [2]:
!ls /content

sample_data  spark-3.0.1-bin-hadoop2.7	spark-3.0.1-bin-hadoop2.7.tgz


In [3]:
!wget -q https://github.com/masfworld/datahack_docker/raw/master/zeppelin/libs/libs-kafka_301.zip --directory-prefix=/content/spark-3.0.1-bin-hadoop2.7/jars/
!unzip -n /content/spark-3.0.1-bin-hadoop2.7/jars/libs-kafka_301.zip -d /content/spark-3.0.1-bin-hadoop2.7/jars/
!ls /content/spark-3.0.1-bin-hadoop2.7/jars/*kafka*

Archive:  /content/spark-3.0.1-bin-hadoop2.7/jars/libs-kafka_301.zip
  inflating: /content/spark-3.0.1-bin-hadoop2.7/jars/ant-1.10.3.jar  
  inflating: /content/spark-3.0.1-bin-hadoop2.7/jars/ant-launcher-1.10.3.jar  
  inflating: /content/spark-3.0.1-bin-hadoop2.7/jars/asm-7.0.jar  
  inflating: /content/spark-3.0.1-bin-hadoop2.7/jars/asm-tree-7.0.jar  
  inflating: /content/spark-3.0.1-bin-hadoop2.7/jars/asm-util-7.0.jar  
  inflating: /content/spark-3.0.1-bin-hadoop2.7/jars/cglib-3.2.10.jar  
  inflating: /content/spark-3.0.1-bin-hadoop2.7/jars/commons-pool2-2.6.2.jar  
  inflating: /content/spark-3.0.1-bin-hadoop2.7/jars/kafka-clients-2.4.1.jar  
  inflating: /content/spark-3.0.1-bin-hadoop2.7/jars/spark-sql-kafka-0-10_2.12-3.0.1.jar  
  inflating: /content/spark-3.0.1-bin-hadoop2.7/jars/spark-token-provider-kafka-0-10_2.12-3.0.1.jar  
/content/spark-3.0.1-bin-hadoop2.7/jars/kafka-clients-2.4.1.jar
/content/spark-3.0.1-bin-hadoop2.7/jars/libs-kafka_301.zip
/content/spark-3.0.1-bin-

Define the environment (Java & Spark homes)

---

In [4]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop2.7"
os.environ["PYSPARK_SUBMIT_ARGS"] = ""

Starting Spark Session and print the version


---


In [5]:
import findspark
findspark.add_packages(["org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.1"])
findspark.add_jars(["/content/spark-3.0.1-bin-hadoop2.7/jars/kafka-clients-2.0.0.jar","/content/spark-3.0.1-bin-hadoop2.7/jars/lz4-java-1.4.1-jar","/content/spark-3.0.1-bin-hadoop2.7/jars/scala-library-2.11.12.jar","/content/spark-3.0.1-bin-hadoop2.7/jars/slf4j-api-1.7.25.jar","/content/spark-3.0.1-bin-hadoop2.7/jars/snappy-java-1.1.7.1.jar","/content/spark-3.0.1-bin-hadoop2.7/jars/spark-sql-kafka-0-10_2.11-2.4.5.jar","/content/spark-3.0.1-bin-hadoop2.7/jars/spark-tags_2.11-2.4.5.jar","/content/spark-3.0.1-bin-hadoop2.7/jars/unused-1.0.0.jar"])
findspark.init("spark-3.0.1-bin-hadoop2.7")# SPARK_HOME

from pyspark.sql import SparkSession

# create the session
spark = SparkSession \
        .builder \
        .master("local[*]") \
        .config("spark.ui.port", "4050") \
        .getOrCreate()

spark.version

'3.0.1'

In [6]:
spark

In [7]:
# For Pandas conversion optimization
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

Creating ngrok tunnel to allow Spark UI (Optional)


In [8]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2021-01-30 09:20:58--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.200.34.95, 3.226.107.193, 52.206.211.104, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.200.34.95|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.13M  36.8MB/s    in 0.4s    

2021-01-30 09:20:58 (36.8 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
Traceback (most recent call last):
  File "<string>", line 1, in <module>
IndexError: list index out of range


# Download Datasets

In [9]:
!mkdir -p /dataset
!wget -q https://github.com/masfworld/datahack_docker/raw/master/zeppelin/data/trades.csv -P /dataset
!wget -q https://github.com/masfworld/datahack_docker/raw/master/zeppelin/data/trades.json -P /dataset
!wget -q https://github.com/masfworld/datahack_docker/raw/master/zeppelin/data/offshore_leaks.edges.csv -P /dataset
!wget -q https://github.com/masfworld/datahack_docker/raw/master/zeppelin/data/offshore_leaks.nodes.address.csv -P /dataset
!wget -q https://github.com/masfworld/datahack_docker/raw/master/zeppelin/data/offshore_leaks.nodes.intermediary.csv -P /dataset
!wget -q https://github.com/masfworld/datahack_docker/raw/master/zeppelin/data/offshore_leaks.nodes.officer.csv -P /dataset
!wget -q https://github.com/masfworld/datahack_docker/raw/master/zeppelin/data/offshore_leaks.nodes.entity.csv -P /dataset
!ls /dataset

offshore_leaks.edges.csv	       offshore_leaks.nodes.officer.csv
offshore_leaks.nodes.address.csv       trades.csv
offshore_leaks.nodes.entity.csv        trades.json
offshore_leaks.nodes.intermediary.csv


# Project 1 - Regulatory Banking Project
---

Input files: /dataset/trades.csv & /dataset/trades.json


In [10]:
df_trade1 = spark \
.read \
.format("csv") \
.option("inferSchema", "true") \
.option("header", "true") \
.load("/dataset/trades.csv")

In [13]:
from pyspark.sql.functions import *

def pnl(open, close):
  if (open is not None and close is not None):
    return (close * 100 / open) - 100
  else:
    return 0

spark.udf.register("pnl_udf", pnl)

<function __main__.pnl>

In [17]:
df_trade1_pnl = (df_trade1
.select("Date", "Open", "Close")
.withColumn("pnl", round(pnl(col("open"), col("close")), 2))
.withColumn("trade", lit("trade_1"))
)

df_trade1_pnl.show()

+----------+-------+------+-----+-------+
|      Date|   Open| Close|  pnl|  trade|
+----------+-------+------+-----+-------+
|2017-12-28| 190.91|189.78|-0.59|trade_1|
|2017-12-27|  190.6|190.19|-0.22|trade_1|
|2017-12-26| 188.53|190.36| 0.97|trade_1|
|2017-12-22|  188.2|188.13|-0.04|trade_1|
|2017-12-21|  187.7|188.08|  0.2|trade_1|
|2017-12-20| 187.14|187.31| 0.09|trade_1|
|2017-12-19|185.235|185.98|  0.4|trade_1|
|2017-12-18| 183.96|184.73| 0.42|trade_1|
|2017-12-15|183.005|182.58|-0.23|trade_1|
|2017-12-14| 183.88|182.13|-0.95|trade_1|
|2017-12-13| 182.01|183.03| 0.56|trade_1|
|2017-12-12| 182.75| 181.8|-0.52|trade_1|
|2017-12-11|  182.9|182.25|-0.36|trade_1|
|2017-12-08|  182.5|183.41|  0.5|trade_1|
|2017-12-07| 180.05| 182.0| 1.08|trade_1|
|2017-12-06| 180.25| 180.8| 0.31|trade_1|
|2017-12-05| 184.79|182.85|-1.05|trade_1|
|2017-12-04| 183.19| 184.9| 0.93|trade_1|
|2017-12-01| 180.32|180.42| 0.06|trade_1|
|2017-11-30| 178.07|179.82| 0.98|trade_1|
+----------+-------+------+-----+-

In [11]:
df_trade2 = spark \
.read \
.format("json") \
.option("inferSchema", "true") \
.option("header", "true") \
.load("/dataset/trades.json")

In [ ]:
df_trade2

# Project 2 - Transactions Notifications

*Hint: https://databricks.com/blog/2017/05/08/event-time-aggregation-watermarking-apache-sparks-structured-streaming.html*

In [21]:
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StructType, StructField, StringType

df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "ec2-34-232-50-77.compute-1.amazonaws.com:9092") \
  .option("subscribe", "transactions") \
  .load()

In [22]:
schema = StructType(
    [
     StructField('Account No', StringType(), True),
     StructField('DATE', StringType(), True),
     StructField('TRANSACTION DETAILS', StringType(), True),
     StructField('CHQ.NO.', StringType(), True),
     StructField('VALUE DATE', StringType(), True),
     StructField(' WITHDRAWAL AMT ', StringType(), True),
     StructField(' DEPOSIT AMT ', StringType(), True),
     StructField('BALANCE AMT', StringType(), True)
    ]
)
df.printSchema()

dataset = df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)", "timestamp") \
    .withColumn("value", from_json("value", schema)) \
    .select(col('key'), col("timestamp"), col('value.*'))

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [23]:
dataset.writeStream \
 .outputMode("update") \
 .format("memory") \
 .option("truncate", "false") \
 .queryName("transactions") \
 .start()

In [29]:
spark.sql("select * from transactions").show(truncate = False)

+----+-----------------------+-------------+---------+-------------------------+-------+----------+----------------+-------------+-----------------+
|key |timestamp              |Account No   |DATE     |TRANSACTION DETAILS      |CHQ.NO.|VALUE DATE| WITHDRAWAL AMT | DEPOSIT AMT |BALANCE AMT      |
+----+-----------------------+-------------+---------+-------------------------+-------+----------+----------------+-------------+-----------------+
|null|2021-01-30 12:05:12.259|409000438611'|23-ago-16|RUPAY POS ACQ SETTL 23081|       |23-ago-16 |                | 1.778,24    | -547.706.347,07 |
|null|2021-01-30 12:05:12.256|409000438611'|20-ago-16|RUPAY POS ACQ SETTL 20081|       |20-ago-16 |                | 5.047,05    | -547.721.273,80 |
|null|2021-01-30 12:05:12.253|409000438611'|18-ago-16|RUPAY POS ACQ SETTL 18081|       |18-ago-16 |                | 13.225,86   | -547.730.739,40 |
|null|2021-01-30 12:05:12.257|409000438611'|22-ago-16|RUPAY POS ACQ SETTL 21081|       |22-ago-16 |       

In [30]:
from pyspark.sql.functions import *

dataset2 = \
  dataset \
    .groupBy(window("timestamp", "5 seconds")) \
    .count()

In [31]:
dataset2.writeStream \
 .outputMode("update") \
 .format("memory") \
 .option("truncate", "false") \
 .queryName("transactions2") \
 .start()

In [36]:
spark.sql("select * from transactions2").show(truncate = False)

+------------------------------------------+-----+
|window                                    |count|
+------------------------------------------+-----+
|[2021-01-30 12:25:35, 2021-01-30 12:25:40]|50   |
|[2021-01-30 12:25:40, 2021-01-30 12:25:45]|50   |
|[2021-01-30 12:25:50, 2021-01-30 12:25:55]|50   |
|[2021-01-30 12:25:45, 2021-01-30 12:25:50]|50   |
|[2021-01-30 12:25:55, 2021-01-30 12:26:00]|50   |
+------------------------------------------+-----+



# Project 3 - Panama Papers

Trace "Spring Song International Co., Ltd." entity with Spark SQL using the following dataset</br>
/dataset/offshore_leaks.nodes.entity.csv </br>
/dataset/offshore_leaks.nodes.intermediary.csv </br>
/dataset/offshore_leaks.edges.csv </br>
/dataset/offshore_leaks.nodes.officer.csv